<a href="https://colab.research.google.com/github/xoro-o/nn_baisc/blob/main/basic_nn_cnn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Imports  
 import torch
 import torch.nn as nn
 import torch.optim as optim # sgd adam etc
 import torch.nn.functional as F # all function which dont take parameters such as ReLU 
 from torch.utils.data import DataLoader
 import torchvision.datasets as datasets
 import torchvision.transforms as transforms
 

In [19]:
class NN(nn.Module):
  def __init__(self,input_size,num_classes):
    super(NN,self).__init__() 
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50,num_classes)

  def forward(self,x):
    x =   F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


model = NN(784,10)
x = torch.randn(64,784)
print(model(x).shape)



torch.Size([64, 10])


In [20]:
class CNN(nn.Module):
  def __init__(self,in_channels = 1,num_classes = 10):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels = in_channels,out_channels = 8,kernel_size = 3,stride=1,padding='same')
    self.pool1 = nn.MaxPool2d(kernel_size = 2,stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 8,out_channels = 16,kernel_size = 3,stride=1,padding='same')
    self.fc1 = nn.Linear(16*7*7,num_classes)
  
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool1(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)
    return x



model = CNN(1,10)
x = torch.randn(64,1,28,28)
print(model(x).shape)

torch.Size([64, 10])


In [21]:
# set device 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs =  1


In [22]:
# Load data
train_dataset = datasets.MNIST(root='dataset/',train=True,transform = transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform = transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle=True)


In [23]:
# initialize network
model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)



In [25]:
#train network
for epoch in range(num_epochs):
  for batch,(data, target) in enumerate(train_loader):
    data = data.to(device=device)
    target = target.to(device=device)

    
    pred = model(data)
    loss = loss_fn(pred,target)

    
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    
    


In [29]:
def check_accuracy(loader,model):
  num_correct = 0
  num_sample = 0 
  model.eval()
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device=device)
      y=y.to(device=device)
      

      scores = model(x)
      _,pred = scores.max(dim=1)
      num_correct += (pred==y).sum()
      num_sample += pred.size(0)
    print(f'accuracy = {float(num_correct)/float(num_sample)*100:.2f}')

In [30]:
check_accuracy(test_loader,model)

accuracy = 97.12
